### Functions

In [ ]:
def get_article_links(search_term, limit = 100):

    driver = webdriver.Firefox()
    
    url_path = f'https://www.investing.com/search/?q={search_term}&tab=news'
    driver.get(url_path)

    links = []

    pos = 0
    while len(driver.find_elements(by=By.CLASS_NAME, value="articleItem")) < limit:
        pos += 500
        driver.execute_script(f'window.scrollTo(0, {pos})')

    article_items = driver.find_elements(by=By.CLASS_NAME, value="articleItem")  
    for article in article_items:
        if article.get_attribute("class") != "js-article-item articleItem     ": 
            link = article.find_element(by=By.CLASS_NAME, value="title")         
            links.append(link.get_attribute("href"))   
            
    driver.quit()
    return links

def get_article_text(link):

    response = requests.get(link)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, 'html.parser')
        article = soup.find('div', class_='WYSIWYG articlePage')
        date = soup.find("div", class_ = "contentSectionDetails")

        # Extracting article text
        script = article.find("script")
        if script != None:
            script.extract()
        img_carousel = article.find('div', class_='imgCarousel')
        if img_carousel != None:
            img_carousel.extract()
        related_instruments_wrapper = article.find('div', class_='relatedInstrumentsWrapper')
        if related_instruments_wrapper != None:
            related_instruments_wrapper.extract()
        paragraph = article.find("p")
        if paragraph != None:
            em = paragraph.find("em")
            if em != None:
                paragraph.extract()
        text_inside_div = article.get_text()
        
        # Extracting article date
        cont_sect_det = soup.find_all("div", class_ = "contentSectionDetails")
        date = cont_sect_det[1].find("span").text
        
        return [text_inside_div.strip(), date[10:-11]]
    
    else:
        return [None, None]
    
def get_article_texts(links, tracker = False):
    results = []
    for i, link in enumerate(links):
        results.append(get_article_text(link))
        if tracker:
            print(i + 1, "/", len(links))
    return results

In [ ]:
# Insert GUI here.  The GUI will take in an input (which is expected to be a stock symbol) and store it in the variable below.

stock_symbol = None

In [ ]:
# Here is where the back-end analysis begins.  First, we get our handful of links given the search term above using the get_article_links() function.
links = get_article_links(stock_symbol)

# Then we take the list of links and get the corresponding text from each article using the get_article_texts() function.
texts = get_article_texts(links)

# Next, we take the list of article texts and feed them into the model to get sentiments.  At this point, the model is not yet built, so I will write some placeholder code here for now.
def sentiment_model(list_of_texts):
    return None
sentiments = sentiment_model(texts)

# Lastly, we can take the mode of the predicted sentiments to get the overall sentiment of the given articles.  Note that there are different ways we can do this; I'm just saying mode for now for simplicity.
from statistics import mode
overall_sentiment = mode(sentiments)

In [ ]:
# Our GUI should output the value of overall_sentiment.